In [6]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import db_dtypes
import snowflake.connector
import math
import boto3
from datetime import datetime
import os

###################################
#### NOTES ########################
#####################################
# NEED TO ADD IN LOGI TO MAKE A TALBE THAT HAS THE RUNTIMES OF EACH TABLE THEN THTE LAST RUN TIME WHERE YOU CHOOSE MAX FROM THE PARTICUAL TABLE THEN SAY GREATER THAN 
#### STORE THIS IN SNOWFLAKE WITH TIMESTAMPS 
#### FILTER IT IN BIG QUERY 
# CLEAR OUT THE LOCAL FILES 
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "surremor-ocf-69d6a4754b0a.json"
key_path = "bigquerykey.json"
credentials = service_account.Credentials.from_service_account_file(key_path)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)
batch_size = 
access_key_id =''
secret_access_key=''
bucket = ''
folder = ''
# Set up AWS S3 client
s3 = boto3.client('s3',
                  aws_access_key_id=access_key_id,
                  aws_secret_access_key=secret_access_key,
                  region_name='us-east-1')

# Snowflake connection details
conn = snowflake.connector.connect(
    user='',
    password='!',
    account='',  # Correct format for the account parameter
    warehouse='',
    database='',
    schema='',
    role=''
)

snowflake_table = ''
staging = ""

cur = conn.cursor()

lastrun = """
select MAX(TIMESTAMP) 
from GOOGLE_BIGQUERY.EVENT_VIEWS.RUNTIME_HST 
WHERE TABLE_NAME ILIKE '%GOOGLE_BIGQUERY.USERS_VIEWS.USERS_FRDV%';
"""

cur.execute(lastrun)

lastrun_result = cur.fetchone()  

if lastrun_result and lastrun_result[0]:
    last_run_time = lastrun_result[0].strftime('%Y-%m-%d')
else:
    last_run_time = None  # Handle case if there is no last run time

query = f"""
SELECT
  *
FROM
  `events data from bigquery`
WHERE
  PARSE_DATE('%Y%m%d', event_date) > {last_run_time}
"""

df = client.query(query).to_dataframe()
# Normalize each nested column
session_traffic_source_normalized = pd.json_normalize(df['session_traffic_source_last_click'])
collected_traffic_source_normalized = pd.json_normalize(df['collected_traffic_source'])
geo_normalized = pd.json_normalize(df['geo'])
device_normalized = pd.json_normalize(df['device'])
privacy_info_normalized = pd.json_normalize(df['privacy_info'])
traffic_source_normalized = pd.json_normalize(df['traffic_source'])
user_properties_normalized = pd.json_normalize(df['user_properties'])
items_normalized = pd.json_normalize(df['items'])

# Join the normalized DataFrames back to the original DataFrame
# Make sure the indexes align properly during the join (i.e., by index)
df_final = df.join([session_traffic_source_normalized, 
                    collected_traffic_source_normalized, 
                    geo_normalized, 
                    device_normalized, 
                    privacy_info_normalized,
                    traffic_source_normalized,
                    items_normalized,
                    user_properties_normalized])

# Drop the original nested columns if necessary
df_final = df_final.drop(columns=['session_traffic_source_last_click', 
                                  'collected_traffic_source', 
                                  'geo', 
                                  'device', 
                                  'privacy_info',
                                  'traffic_source',
                                  'items',
                                  'user_properties'])


df_exploded = df_final.explode('event_params')
df_exploded['param_key'] = df_exploded['event_params'].apply(lambda x: x['key'] if isinstance(x, dict) else None)
df_exploded['param_value'] = df_exploded['event_params'].apply(lambda x: x['value'] if isinstance(x, dict) else None)
df_exploded['value_string'] = df_exploded['param_value'].apply(lambda x: x.get('string_value') if isinstance(x, dict) else None)
df_exploded['value_int'] = df_exploded['param_value'].apply(lambda x: x.get('int_value') if isinstance(x, dict) else None)
df_exploded['value_float'] = df_exploded['param_value'].apply(lambda x: x.get('float_value') if isinstance(x, dict) else None)
df_final = df_exploded.drop(columns=['event_params', 'param_value'])
df_final.columns = df_final.columns.str.replace('.', '_', regex=False)
#df_final.to_csv('events.csv', index = False)

numeric_columns = df_final.select_dtypes(include=['float64', 'int64']).columns
df_final[numeric_columns] = df_final[numeric_columns].fillna(0)
string_columns = df_final.select_dtypes(include=['object']).columns
df_final[string_columns] = df_final[string_columns].fillna('Unknown')
boolean_columns = df_final.select_dtypes(include=['bool']).columns
df_final[boolean_columns] = df_final[boolean_columns].fillna(False)
df_final.dropna(axis=1, how='all', inplace=True)

df_final = df_final.astype({
    'event_date': 'object',   
    'event_timestamp': 'object',   
    'event_name': 'object',   
    'event_previous_timestamp': 'object',   
    'event_value_in_usd': 'object',   
    'event_bundle_sequence_id': 'object',   
    'event_server_timestamp_offset': 'object',   
    'user_id': 'object',   
    'user_pseudo_id': 'object',   
    'user_first_touch_timestamp': 'object',   
    'user_ltv': 'object',   
    'app_info': 'object',   
    'stream_id': 'object',   
    'platform': 'object',   
    'event_dimensions': 'object',   
    'ecommerce': 'object',   
    'is_active_user': 'object',   
    'batch_event_index': 'object',   
    'batch_page_id': 'object',   
    'batch_ordering_id': 'object',   
    'google_ads_campaign': 'object',   
    'manual_campaign_campaign_id': 'object',   
    'manual_campaign_campaign_name': 'object',   
    'manual_campaign_source': 'object',   
    'manual_campaign_medium': 'object',   
    'manual_campaign_term': 'object',   
    'manual_campaign_content': 'object',   
    'manual_campaign_source_platform': 'object',   
    'manual_campaign_creative_format': 'object',   
    'manual_campaign_marketing_tactic': 'object',   
    'manual_campaign_id': 'object',   
    'manual_campaign_name': 'object',   
    'manual_source': 'object',   
    'manual_medium': 'object',   
    'manual_term': 'object',   
    'manual_content': 'object',   
    'manual_source_platform': 'object',   
    'manual_creative_format': 'object',   
    'manual_marketing_tactic': 'object',   
    'gclid': 'object',   
    'dclid': 'object',   
    'srsltid': 'object',   
    'city': 'object',   
    'country': 'object',   
    'continent': 'object',   
    'region': 'object',   
    'sub_continent': 'object',   
    'metro': 'object',   
    'category': 'object',   
    'mobile_brand_name': 'object',   
    'mobile_model_name': 'object',   
    'mobile_marketing_name': 'object',   
    'mobile_os_hardware_model': 'object',   
    'operating_system': 'object',   
    'operating_system_version': 'object',   
    'vendor_id': 'object',   
    'advertising_id': 'object',   
    'language': 'object',   
    'is_limited_ad_tracking': 'object',   
    'time_zone_offset_seconds': 'object',   
    'browser': 'object',   
    'browser_version': 'object',   
    'web_info_browser': 'object',   
    'web_info_browser_version': 'object',   
    'web_info_hostname': 'object',   
    'analytics_storage': 'object',   
    'ads_storage': 'object',   
    'uses_transient_token': 'object',   
    'name': 'object',   
    'medium': 'object',   
    'source': 'object',   
    'param_key': 'object',   
    'value_string': 'object',   
    'value_int': 'object',   
    'value_float': 'object'   
})

num_batches = len(df_final) // batch_size + 1

# Loop through and upload batches
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(df_final))
    
    # Slice DataFrame into a batch
    batch_data = df_final[start_idx:end_idx]
    
    # Save batch to a temporary CSV file
    file_name = f'batch_{i+1}.csv'
    batch_data.to_csv(file_name, index=False)
    
    # Upload CSV to S3
    s3.upload_file(file_name, bucket, f'{folder}/' + file_name)

    print(f"Uploaded {file_name} to S3.")

cur = conn.cursor()

# Load data from S3 into Snowflake table
copy_command = f"""
COPY INTO EVENTS_FRDV
FROM 's3://{bucket}/'
CREDENTIALS = (AWS_KEY_ID='{access_key_id}' AWS_SECRET_KEY='{secret_access_key}')
FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY = '"');
"""


merge = f"""MERGE INTO {snowflake_table} AS tgt
USING {staging}AS src
ON tgt.EVENT_DATE = src.EVENT_DATE
   AND tgt.USER_ID = src.USER_ID
   AND tgt.EVENT_NAME = src.EVENT_NAME
WHEN MATCHED THEN
    UPDATE SET
        tgt.EVENT_TIMESTAMP = src.EVENT_TIMESTAMP,
        tgt.EVENT_PREVIOUS_TIMESTAMP = src.EVENT_PREVIOUS_TIMESTAMP,
        tgt.EVENT_VALUE_IN_USD = src.EVENT_VALUE_IN_USD,
        tgt.EVENT_BUNDLE_SEQUENCE_ID = src.EVENT_BUNDLE_SEQUENCE_ID,
        tgt.EVENT_SERVER_TIMESTAMP_OFFSET = src.EVENT_SERVER_TIMESTAMP_OFFSET,
        tgt.USER_PSEUDO_ID = src.USER_PSEUDO_ID,
        tgt.USER_FIRST_TOUCH_TIMESTAMP = src.USER_FIRST_TOUCH_TIMESTAMP,
        tgt.USER_LTV = src.USER_LTV,
        tgt.APP_INFO = src.APP_INFO,
        tgt.STREAM_ID = src.STREAM_ID,
        tgt.PLATFORM = src.PLATFORM,
        tgt.EVENT_DIMENSIONS = src.EVENT_DIMENSIONS,
        tgt.ECOMMERCE = src.ECOMMERCE,
        tgt.IS_ACTIVE_USER = src.IS_ACTIVE_USER,
        tgt.BATCH_EVENT_INDEX = src.BATCH_EVENT_INDEX,
        tgt.BATCH_PAGE_ID = src.BATCH_PAGE_ID,
        tgt.BATCH_ORDERING_ID = src.BATCH_ORDERING_ID,
        tgt.GOOGLE_ADS_CAMPAIGN = src.GOOGLE_ADS_CAMPAIGN,
        tgt.MANUAL_CAMPAIGN_CAMPAIGN_ID = src.MANUAL_CAMPAIGN_CAMPAIGN_ID,
        tgt.MANUAL_CAMPAIGN_CAMPAIGN_NAME = src.MANUAL_CAMPAIGN_CAMPAIGN_NAME,
        tgt.MANUAL_CAMPAIGN_SOURCE = src.MANUAL_CAMPAIGN_SOURCE,
        tgt.MANUAL_CAMPAIGN_MEDIUM = src.MANUAL_CAMPAIGN_MEDIUM,
        tgt.MANUAL_CAMPAIGN_TERM = src.MANUAL_CAMPAIGN_TERM,
        tgt.MANUAL_CAMPAIGNCONTENT = src.MANUAL_CAMPAIGNCONTENT,
        tgt.MANUAL_CAMPAIGN_SOURCE_PLATFORM = src.MANUAL_CAMPAIGN_SOURCE_PLATFORM,
        tgt.MANUAL_CAMPAIGN_CREATIVE_FORMAT = src.MANUAL_CAMPAIGN_CREATIVE_FORMAT,
        tgt.MANUAL_CAMPAIGN_MARKETING_TACTIC = src.MANUAL_CAMPAIGN_MARKETING_TACTIC,
        tgt.MANUAL_CAMPAIGN_ID = src.MANUAL_CAMPAIGN_ID,
        tgt.MANUAL_CAMPAIGN_NAME = src.MANUAL_CAMPAIGN_NAME,
        tgt.MANUAL_SOURCE = src.MANUAL_SOURCE,
        tgt.MANUAL_MEDIUM = src.MANUAL_MEDIUM,
        tgt.MANUAL_TERM = src.MANUAL_TERM,
        tgt.MANUAL_CONTENT = src.MANUAL_CONTENT,
        tgt.MANUAL_SOURCE_PLATFORM = src.MANUAL_SOURCE_PLATFORM,
        tgt.MANUAL_CREATIVE_FORMAT = src.MANUAL_CREATIVE_FORMAT,
        tgt.MANUAL_MARKETING_TACTIC = src.MANUAL_MARKETING_TACTIC,
        tgt.GCLID = src.GCLID,
        tgt.DCLID = src.DCLID,
        tgt.SRSLTID = src.SRSLTID,
        tgt.CITY = src.CITY,
        tgt.COUNTRY = src.COUNTRY,
        tgt.CONTINENT = src.CONTINENT,
        tgt.REGION = src.REGION,
        tgt.SUB_CONTINENT = src.SUB_CONTINENT,
        tgt.METRO = src.METRO,
        tgt.CATEGORY = src.CATEGORY,
        tgt.MOBILE_BRAND_NAME = src.MOBILE_BRAND_NAME,
        tgt.MOBILE_MODEL_NAME = src.MOBILE_MODEL_NAME,
        tgt.MOBILE_MARKETING_NAME = src.MOBILE_MARKETING_NAME,
        tgt.MOBILE_OS_HARDWARE_MODEL = src.MOBILE_OS_HARDWARE_MODEL,
        tgt.OPERATING_SYSTEM = src.OPERATING_SYSTEM,
        tgt.OPERATING_SYSTEM_VERSION = src.OPERATING_SYSTEM_VERSION,
        tgt.VENDOR_ID = src.VENDOR_ID,
        tgt.ADVERTISING_ID = src.ADVERTISING_ID,
        tgt.LANGUAGE = src.LANGUAGE,
        tgt.IS_LIMITED_AD_TRACKING = src.IS_LIMITED_AD_TRACKING,
        tgt.TIME_ZONE_OFFSET_SECONDS = src.TIME_ZONE_OFFSET_SECONDS,
        tgt.BROWSER = src.BROWSER,
        tgt.BROWSER_VERSION = src.BROWSER_VERSION,
        tgt.WEB_INFO_BROWSER = src.WEB_INFO_BROWSER,
        tgt.WEB_INFO_BROWSER_VERSION = src.WEB_INFO_BROWSER_VERSION,
        tgt.WEB_INFO_HOSTNAME = src.WEB_INFO_HOSTNAME,
        tgt.ANALYTICS_STORAGE = src.ANALYTICS_STORAGE,
        tgt.ADS_STORAGE = src.ADS_STORAGE,
        tgt.USES_TRANSIENT_TOKEN = src.USES_TRANSIENT_TOKEN,
        tgt.NAME = src.NAME,
        tgt.MEDIUM = src.MEDIUM,
        tgt.SOURCE = src.SOURCE,
        tgt.PARAM_KEY = src.PARAM_KEY,
        tgt.VALUE_STRING = src.VALUE_STRING,
        tgt.VALUE_INT = src.VALUE_INT,
        tgt.VALUE_FLOAT = src.VALUE_FLOAT
WHEN NOT MATCHED THEN
    INSERT (
        EVENT_DATE,
        EVENT_TIMESTAMP,
        EVENT_NAME,
        EVENT_PREVIOUS_TIMESTAMP,
        EVENT_VALUE_IN_USD,
        EVENT_BUNDLE_SEQUENCE_ID,
        EVENT_SERVER_TIMESTAMP_OFFSET,
        USER_ID,
        USER_PSEUDO_ID,
        USER_FIRST_TOUCH_TIMESTAMP,
        USER_LTV,
        APP_INFO,
        STREAM_ID,
        PLATFORM,
        EVENT_DIMENSIONS,
        ECOMMERCE,
        IS_ACTIVE_USER,
        BATCH_EVENT_INDEX,
        BATCH_PAGE_ID,
        BATCH_ORDERING_ID,
        GOOGLE_ADS_CAMPAIGN,
        MANUAL_CAMPAIGN_CAMPAIGN_ID,
        MANUAL_CAMPAIGN_CAMPAIGN_NAME,
        MANUAL_CAMPAIGN_SOURCE,
        MANUAL_CAMPAIGN_MEDIUM,
        MANUAL_CAMPAIGN_TERM,
        MANUAL_CAMPAIGNCONTENT,
        MANUAL_CAMPAIGN_SOURCE_PLATFORM,
        MANUAL_CAMPAIGN_CREATIVE_FORMAT,
        MANUAL_CAMPAIGN_MARKETING_TACTIC,
        MANUAL_CAMPAIGN_ID,
        MANUAL_CAMPAIGN_NAME,
        MANUAL_SOURCE,
        MANUAL_MEDIUM,
        MANUAL_TERM,
        MANUAL_CONTENT,
        MANUAL_SOURCE_PLATFORM,
        MANUAL_CREATIVE_FORMAT,
        MANUAL_MARKETING_TACTIC,
        GCLID,
        DCLID,
        SRSLTID,
        CITY,
        COUNTRY,
        CONTINENT,
        REGION,
        SUB_CONTINENT,
        METRO,
        CATEGORY,
        MOBILE_BRAND_NAME,
        MOBILE_MODEL_NAME,
        MOBILE_MARKETING_NAME,
        MOBILE_OS_HARDWARE_MODEL,
        OPERATING_SYSTEM,
        OPERATING_SYSTEM_VERSION,
        VENDOR_ID,
        ADVERTISING_ID,
        LANGUAGE,
        IS_LIMITED_AD_TRACKING,
        TIME_ZONE_OFFSET_SECONDS,
        BROWSER,
        BROWSER_VERSION,
        WEB_INFO_BROWSER,
        WEB_INFO_BROWSER_VERSION,
        WEB_INFO_HOSTNAME,
        ANALYTICS_STORAGE,
        ADS_STORAGE,
        USES_TRANSIENT_TOKEN,
        NAME,
        MEDIUM,
        SOURCE,
        PARAM_KEY,
        VALUE_STRING,
        VALUE_INT,
        VALUE_FLOAT
    )
    VALUES (
        src.EVENT_DATE,
        src.EVENT_TIMESTAMP,
        src.EVENT_NAME,
        src.EVENT_PREVIOUS_TIMESTAMP,
        src.EVENT_VALUE_IN_USD,
        src.EVENT_BUNDLE_SEQUENCE_ID,
        src.EVENT_SERVER_TIMESTAMP_OFFSET,
        src.USER_ID,
        src.USER_PSEUDO_ID,
        src.USER_FIRST_TOUCH_TIMESTAMP,
        src.USER_LTV,
        src.APP_INFO,
        src.STREAM_ID,
        src.PLATFORM,
        src.EVENT_DIMENSIONS,
        src.ECOMMERCE,
        src.IS_ACTIVE_USER,
        src.BATCH_EVENT_INDEX,
        src.BATCH_PAGE_ID,
        src.BATCH_ORDERING_ID,
        src.GOOGLE_ADS_CAMPAIGN,
        src.MANUAL_CAMPAIGN_CAMPAIGN_ID,
        src.MANUAL_CAMPAIGN_CAMPAIGN_NAME,
        src.MANUAL_CAMPAIGN_SOURCE,
        src.MANUAL_CAMPAIGN_MEDIUM,
        src.MANUAL_CAMPAIGN_TERM,
        src.MANUAL_CAMPAIGNCONTENT,
        src.MANUAL_CAMPAIGN_SOURCE_PLATFORM,
        src.MANUAL_CAMPAIGN_CREATIVE_FORMAT,
        src.MANUAL_CAMPAIGN_MARKETING_TACTIC,
        src.MANUAL_CAMPAIGN_ID,
        src.MANUAL_CAMPAIGN_NAME,
        src.MANUAL_SOURCE,
        src.MANUAL_MEDIUM,
        src.MANUAL_TERM,
        src.MANUAL_CONTENT,
        src.MANUAL_SOURCE_PLATFORM,
        src.MANUAL_CREATIVE_FORMAT,
        src.MANUAL_MARKETING_TACTIC,
        src.GCLID,
        src.DCLID,
        src.SRSLTID,
        src.CITY,
        src.COUNTRY,
        src.CONTINENT,
        src.REGION,
        src.SUB_CONTINENT,
        src.METRO,
        src.CATEGORY,
        src.MOBILE_BRAND_NAME,
        src.MOBILE_MODEL_NAME,
        src.MOBILE_MARKETING_NAME,
        src.MOBILE_OS_HARDWARE_MODEL,
        src.OPERATING_SYSTEM,
        src.OPERATING_SYSTEM_VERSION,
        src.VENDOR_ID,
        src.ADVERTISING_ID,
        src.LANGUAGE,
        src.IS_LIMITED_AD_TRACKING,
        src.TIME_ZONE_OFFSET_SECONDS,
        src.BROWSER,
        src.BROWSER_VERSION,
        src.WEB_INFO_BROWSER,
        src.WEB_INFO_BROWSER_VERSION,
        src.WEB_INFO_HOSTNAME,
        src.ANALYTICS_STORAGE,
        src.ADS_STORAGE,
        src.USES_TRANSIENT_TOKEN,
        src.NAME,
        src.MEDIUM,
        src.SOURCE,
        src.PARAM_KEY,
        src.VALUE_STRING,
        src.VALUE_INT,
        src.VALUE_FLOAT);"""

truncate = f"truncate {staging};"
cur.execute(truncate)

cur.execute(copy_command)
tracking_command = """
INSERT INTO GOOGLE_BIGQUERY.EVENT_VIEWS.RUNTIME_HST (table_name, table_update, timestamp)
VALUES (%s, %s, %s);
"""
cur.execute(tracking_command, (snowflake_table, 'INSERT', datetime.now()))

response = s3.list_objects_v2(Bucket=f'{bucket}', Prefix=f'{folder}/')

if 'Contents' in response:
    for obj in response['Contents']:
        print(f"Deleting {obj['Key']}")
        s3.delete_object(Bucket=bucket, Key=obj['Key'])
else:
    print("No objects found in the folder.")
print("Folder cleared.")
conn.commit()
conn.close()

Uploaded batch_1.csv to S3.
Deleting Events/batch_1.csv
Folder cleared.
